In [188]:
import pandas as pd
import numpy as np
from itertools import combinations, product
from functools import reduce
from collections import Iterable
    
# use for vertify 
from mlxtend.frequent_patterns import apriori, association_rules

In [189]:
real_dataset_path = "/home/sokhorn/sokhorn/dataSet/data/Online Retail.csv"
testing_dataset_path = '/home/sokhorn/sokhorn/dataSet/data/sample_data_set.csv'

sample_dataset = pd.read_csv(
    testing_dataset_path, sep=',', usecols=[
        'InvoiceNo',
        'StockCode',
        'Quantity'
    ])
sample_dataset


,InvoiceNo,StockCode,Quantity
0,536365,85123A,6
1,536365,71053,6
2,536365,84406B,8
3,536365,84029G,6
4,536365,84029E,6
5,536365,22752,2
6,536365,21730,6
7,536366,22633,6
8,536366,22632,6
9,536367,84879,32


In [190]:
item_sets = (
    sample_dataset.groupby(['InvoiceNo', 'StockCode', ])['Quantity']
    .sum().unstack().reset_index().fillna(0)
    .set_index("InvoiceNo")
)
# item_sets = item_sets.iloc[:, :10]
item_sets = item_sets.head(1000)

In [191]:
en_code = lambda x : 1 if x > 0  else 0
item_sets = item_sets.applymap(en_code)
item_sets

StockCode,21730,22632,22633,22745,22748,22749,22752,71053,84029E,84029G,84406B,84879,85123A
InvoiceNo,,,,,,,,,,,,,
536365,1,0,0,0,0,0,1,1,1,1,1,0,1
536366,0,1,1,0,0,0,0,0,0,0,0,0,0
536367,0,0,0,1,1,1,0,0,0,0,0,1,0
536368,0,0,0,1,1,1,0,0,0,0,0,1,0
536369,0,0,0,1,1,1,0,0,0,0,0,1,0
536370,0,0,0,0,0,1,0,0,0,0,0,0,0
536371,0,0,0,0,0,1,0,0,0,0,0,0,0


**Generating RC column for each tranctions**

In [192]:
def rc(item_np):
    np_hash = {}
    item_rc = []
    for item in item_np:
        key = " ".join(map(str, item))
        if np_hash.get(key):
            np_hash[key] += 1
        else:
            np_hash[key] = 1
    
    for item in np_hash:
        values =  list(map(int,  item.split()))
        values.append(np_hash[item])
        item_rc.append(values)
    return item_rc


In [193]:
rc_np = np.array(rc(item_sets.values))
df_rc =  pd.DataFrame(rc_np)
columns = list(item_sets.columns)
columns.append("RC")
df_rc.set_axis(columns, inplace=True  , axis=1 )
RC = df_rc['RC']
df_rc.drop(['RC'], axis=1, inplace=True)
df_rc

,21730,22632,22633,22745,22748,22749,22752,71053,84029E,84029G,84406B,84879,85123A
0,1,0,0,0,0,0,1,1,1,1,1,0,1
1,0,1,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,1,1,0,0,0,0,0,1,0
3,0,0,0,0,0,1,0,0,0,0,0,0,0


In [194]:
# reduce(function, input)
def support_k_itemst(k_itemst):
    s = 0
    for i in range(len(k_itemst)):
        s += reduce(lambda a, b: a & b, k_itemst[i] & RC[i])
    return s

In [195]:
def count_ocurence(itemset, Tranctions):
    count = 0
    for i in range(len(Tranctions)):
        if set(itemset).issubset(set(Tranctions[i])):
            count += 1
    return count

In [196]:
def join_set_item(set_of_its, order_column_name):
    C = []
    k = len(set_of_its[0])  # get K itmset size
    for i in range(len(set_of_its)):
        set_i = set_of_its[i]
        for j in range(i + 1, len(set_of_its)):
            set_j = set_of_its[j]
            if(k > 3):
                if(set_i[::k-1] == set_j[::k-1]):
                    it_out = join_two_itemsets(
                        set_i, set_j, order_column_name)
                    if(len(it_out)) > 0:
                        C.append(it_out)
            else:
                it_out = join_two_itemsets(
                    set_i, set_j, order_column_name)
                if(len(it_out)) > 0:
                    C.append(it_out)

    return C


In [197]:
'71053', '84029E', '84029G', '84406B','85123A'

('71053', '84029E', '84029G', '84406B', '85123A')

In [198]:
df_rc[['21730', '22752', ]]

,21730,22752
0,1,1
1,0,0
2,0,0
3,0,0


In [199]:
support_k_itemst(df_rc[['21730', '22752','84029G' ]].values)

1

In [200]:
df_rc[['21730', '22752']]

,21730,22752
0,1,1
1,0,0
2,0,0
3,0,0


**Fk-1 x Fk-1**

In [201]:
def apriori_gen(Lk, order):
    k = len(Lk)
    result_list = []
    for i in range(k):
        L1 = list(Lk[i])[::k-1]
        for j in range(i + 1, k):
            L2 = list(Lk[j])[::k-1]
            L1.sort(key=lambda x : order.index(x))
            L2.sort(key=lambda x : order.index(x))
            if L1 == L2:
                result_list.append(np.union1d(Lk[i], Lk[j]))
    return result_list

In [202]:
def join_two_itemsets(it1, it2, order):
    it1.sort(key=lambda x: order.index(x))
    it2.sort(key=lambda x: order.index(x))

    for i in range(len(it1) - 1):  # check befor the last one
        if it1[i] != it2[i]:
            return []

    if order.index(it1[-1]) < order.index(it2[-1]):
        return [it1] + [it2[-1]]
    return []


**Join Itemset**

In [203]:
def mergeKItemIntoOne(ab):
    result = []
    if(len(ab) != 0):
        if(len(ab[0]) == 1):
            return ab
        else:
            for item in ab:
                res = item[0] + item[1].split()
                result.append(res)
        return result


In [204]:
def get_frequent(itemesets, min_support, prev_discard):
    L = []
    support_count = []
    new_discard = []
    column_items = mergeKItemIntoOne(itemesets)

    k = len(prev_discard)
    for i in range(len(itemesets)):
        discard_before = False
        item = itemesets[i] 
        result = []
        for i_item in item:
            if isinstance(i_item, list):
                for j in i_item:
                    result.append(j)
            else:
                result.append(i_item)

        if k > 0:
            for it in prev_discard[k]:
                if set(it).issubset(set(result)):
                    discard_before = True
                    break
        
        if not discard_before:
            # print(f'item {mergeKItemIntoOne(item)}')
            count = support_k_itemst(df_rc[column_items[i]].values)
            if count >= min_support:
                L.append(result)
                support_count.append(count)
            else:
                new_discard.append(result)
    return L, support_count, new_discard

In [205]:
def generate_next_itemset(L):
    k = L.shape[-1] + 1
    print(len(L))
    aa =  np.array(list(combinations(L, k)))
    col_name  =  np.hstack((aa[:, 0, :], aa[:, 1, :])) 
    return col_name

**Generating 1 itemse base on support count of row**

In [206]:
C = {}
L = {}
k_items = []
Discard = {}
itemset_size = 1
min_support = 1
Discard.update({itemset_size : []})

#### Generating 1 itemset

In [207]:
df_rc

,21730,22632,22633,22745,22748,22749,22752,71053,84029E,84029G,84406B,84879,85123A
0,1,0,0,0,0,0,1,1,1,1,1,0,1
1,0,1,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,1,1,0,0,0,0,0,1,0
3,0,0,0,0,0,1,0,0,0,0,0,0,0


In [208]:
# remove each column who support count are less than min_support for 1 itemsets
rc_1_itemset = df_rc.sum(axis=0)
cut_our_cols = rc_1_itemset.loc[lambda x: x < min_support].index # remove all column which sum of row are ness than user defind support threshold 
cut_our_cols
df_rc.drop(labels=cut_our_cols, axis=1, inplace=True) # cut our every itemset which are sum of each row are less than user defind support 
df_rc

,21730,22632,22633,22745,22748,22749,22752,71053,84029E,84029G,84406B,84879,85123A
0,1,0,0,0,0,0,1,1,1,1,1,0,1
1,0,1,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,1,1,0,0,0,0,0,1,0
3,0,0,0,0,0,1,0,0,0,0,0,0,0


In [210]:
C.update({itemset_size: np.reshape(list(df_rc.columns), (-1, 1))})

In [211]:
support_count = {}
f, supp, new_discard = get_frequent(
    C[itemset_size], min_support, Discard
)
Discard.update({itemset_size: new_discard})
L.update({itemset_size: f})
support_count.update({itemset_size: supp})

In [212]:
k = itemset_size + 1
while True:
    try:
      C.update({k: join_set_item(L[k - 1], list( df_rc.columns))})
      f, supp, new_discard = get_frequent(C[k], min_support, Discard)
      L.update({k: f})
      Discard.update({k: new_discard})
      support_count.update({k: supp})
      if(len(L[k]) == 0):
          break
      k += 1
    except:
      print("Hello error ")

In [213]:
L

{1: [['21730'],
  ['22632'],
  ['22633'],
  ['22745'],
  ['22748'],
  ['22749'],
  ['22752'],
  ['71053'],
  ['84029E'],
  ['84029G'],
  ['84406B'],
  ['84879'],
  ['85123A']],
 2: [['21730', '22752'],
  ['21730', '71053'],
  ['21730', '84029E'],
  ['21730', '84029G'],
  ['21730', '84406B'],
  ['21730', '85123A'],
  ['22632', '22633'],
  ['22745', '22748'],
  ['22745', '22749'],
  ['22745', '84879'],
  ['22748', '22749'],
  ['22748', '84879'],
  ['22749', '84879'],
  ['22752', '71053'],
  ['22752', '84029E'],
  ['22752', '84029G'],
  ['22752', '84406B'],
  ['22752', '85123A'],
  ['71053', '84029E'],
  ['71053', '84029G'],
  ['71053', '84406B'],
  ['71053', '85123A'],
  ['84029E', '84029G'],
  ['84029E', '84406B'],
  ['84029E', '85123A'],
  ['84029G', '84406B'],
  ['84029G', '85123A'],
  ['84406B', '85123A']],
 3: [['21730', '22752', '71053'],
  ['21730', '22752', '84029E'],
  ['21730', '22752', '84029G'],
  ['21730', '22752', '84406B'],
  ['21730', '22752', '85123A'],
  ['21730', '7105

In [214]:
# itemset = apriori(df_rc, min_support=min_support / len(item_sets), use_colnames=True)
# itemset

In [215]:
# rules = association_rules(itemset, min_threshold=1)
# rules

In [216]:
RC

0    1
1    1
2    3
3    2
Name: RC, dtype: int64

In [217]:
df_rc

,21730,22632,22633,22745,22748,22749,22752,71053,84029E,84029G,84406B,84879,85123A
0,1,0,0,0,0,0,1,1,1,1,1,0,1
1,0,1,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,1,1,0,0,0,0,0,1,0
3,0,0,0,0,0,1,0,0,0,0,0,0,0


#### Testing code 

In [218]:
# support_k_itemst(df_rc[col_name[0]].values)

In [219]:
# supp = support_k_itemst(df_rc[col_name].values)
# supp

In [220]:
col_2_item =  np.array(list(product(L[3], L[1]))).T.reshape(-1, 2)
col_2_item

/tmp/ipykernel_16611/2707254593.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  col_2_item =  np.array(list(product(L[3], L[1]))).T.reshape(-1, 2)


array([[list(['21730', '22752', '71053']),
        list(['21730', '22752', '71053'])],
       [list(['21730', '22752', '71053']),
        list(['21730', '22752', '71053'])],
       [list(['21730', '22752', '71053']),
        list(['21730', '22752', '71053'])],
       ...,
       [list(['71053']), list(['84029E'])],
       [list(['84029G']), list(['84406B'])],
       [list(['84879']), list(['85123A'])]], dtype=object)

In [221]:
L[1]

[['21730'],
 ['22632'],
 ['22633'],
 ['22745'],
 ['22748'],
 ['22749'],
 ['22752'],
 ['71053'],
 ['84029E'],
 ['84029G'],
 ['84406B'],
 ['84879'],
 ['85123A']]

In [222]:
join_set_item(L[1], list(df_rc.columns))

[[['21730'], '22632'],
 [['21730'], '22633'],
 [['21730'], '22745'],
 [['21730'], '22748'],
 [['21730'], '22749'],
 [['21730'], '22752'],
 [['21730'], '71053'],
 [['21730'], '84029E'],
 [['21730'], '84029G'],
 [['21730'], '84406B'],
 [['21730'], '84879'],
 [['21730'], '85123A'],
 [['22632'], '22633'],
 [['22632'], '22745'],
 [['22632'], '22748'],
 [['22632'], '22749'],
 [['22632'], '22752'],
 [['22632'], '71053'],
 [['22632'], '84029E'],
 [['22632'], '84029G'],
 [['22632'], '84406B'],
 [['22632'], '84879'],
 [['22632'], '85123A'],
 [['22633'], '22745'],
 [['22633'], '22748'],
 [['22633'], '22749'],
 [['22633'], '22752'],
 [['22633'], '71053'],
 [['22633'], '84029E'],
 [['22633'], '84029G'],
 [['22633'], '84406B'],
 [['22633'], '84879'],
 [['22633'], '85123A'],
 [['22745'], '22748'],
 [['22745'], '22749'],
 [['22745'], '22752'],
 [['22745'], '71053'],
 [['22745'], '84029E'],
 [['22745'], '84029G'],
 [['22745'], '84406B'],
 [['22745'], '84879'],
 [['22745'], '85123A'],
 [['22748'], '2274

In [223]:
df_rc

,21730,22632,22633,22745,22748,22749,22752,71053,84029E,84029G,84406B,84879,85123A
0,1,0,0,0,0,0,1,1,1,1,1,0,1
1,0,1,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,1,1,0,0,0,0,0,1,0
3,0,0,0,0,0,1,0,0,0,0,0,0,0


In [224]:
# supp_2_itemset =  [support_k_itemst(df_rc[col].values) for col in col_2_item]
# supp_2_itemset

sort_by_index =  lambda cols : cols.index   
print(f"List before Sort {L[1]}")
l =  list(L[1]).sort(key =sort_by_index(list(df_rc.columns)))
print(f"List after Sort {l}")

### Debug code

In [225]:
L

{1: [['21730'],
  ['22632'],
  ['22633'],
  ['22745'],
  ['22748'],
  ['22749'],
  ['22752'],
  ['71053'],
  ['84029E'],
  ['84029G'],
  ['84406B'],
  ['84879'],
  ['85123A']],
 2: [['21730', '22752'],
  ['21730', '71053'],
  ['21730', '84029E'],
  ['21730', '84029G'],
  ['21730', '84406B'],
  ['21730', '85123A'],
  ['22632', '22633'],
  ['22745', '22748'],
  ['22745', '22749'],
  ['22745', '84879'],
  ['22748', '22749'],
  ['22748', '84879'],
  ['22749', '84879'],
  ['22752', '71053'],
  ['22752', '84029E'],
  ['22752', '84029G'],
  ['22752', '84406B'],
  ['22752', '85123A'],
  ['71053', '84029E'],
  ['71053', '84029G'],
  ['71053', '84406B'],
  ['71053', '85123A'],
  ['84029E', '84029G'],
  ['84029E', '84406B'],
  ['84029E', '85123A'],
  ['84029G', '84406B'],
  ['84029G', '85123A'],
  ['84406B', '85123A']],
 3: [['21730', '22752', '71053'],
  ['21730', '22752', '84029E'],
  ['21730', '22752', '84029G'],
  ['21730', '22752', '84406B'],
  ['21730', '22752', '85123A'],
  ['21730', '7105

In [226]:
Discard

{1: [],
 2: [['21730', '22632'],
  ['21730', '22633'],
  ['21730', '22745'],
  ['21730', '22748'],
  ['21730', '22749'],
  ['21730', '84879'],
  ['22632', '22745'],
  ['22632', '22748'],
  ['22632', '22749'],
  ['22632', '22752'],
  ['22632', '71053'],
  ['22632', '84029E'],
  ['22632', '84029G'],
  ['22632', '84406B'],
  ['22632', '84879'],
  ['22632', '85123A'],
  ['22633', '22745'],
  ['22633', '22748'],
  ['22633', '22749'],
  ['22633', '22752'],
  ['22633', '71053'],
  ['22633', '84029E'],
  ['22633', '84029G'],
  ['22633', '84406B'],
  ['22633', '84879'],
  ['22633', '85123A'],
  ['22745', '22752'],
  ['22745', '71053'],
  ['22745', '84029E'],
  ['22745', '84029G'],
  ['22745', '84406B'],
  ['22745', '85123A'],
  ['22748', '22752'],
  ['22748', '71053'],
  ['22748', '84029E'],
  ['22748', '84029G'],
  ['22748', '84406B'],
  ['22748', '85123A'],
  ['22749', '22752'],
  ['22749', '71053'],
  ['22749', '84029E'],
  ['22749', '84029G'],
  ['22749', '84406B'],
  ['22749', '85123A'],
 

In [227]:
l_1 =  np.array(L[1])

In [228]:
l_2 = [x for x in range(len(l_1))]
l_2

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [229]:
L[3][:15]

[['21730', '22752', '71053'],
 ['21730', '22752', '84029E'],
 ['21730', '22752', '84029G'],
 ['21730', '22752', '84406B'],
 ['21730', '22752', '85123A'],
 ['21730', '71053', '84029E'],
 ['21730', '71053', '84029G'],
 ['21730', '71053', '84406B'],
 ['21730', '71053', '85123A'],
 ['21730', '84029E', '84029G'],
 ['21730', '84029E', '84406B'],
 ['21730', '84029E', '85123A'],
 ['21730', '84029G', '84406B'],
 ['21730', '84029G', '85123A'],
 ['21730', '84406B', '85123A']]

In [230]:
np.array(np.meshgrid(L[3], L[3]))

array([[['21730', '22752', '71053', ..., '84029G', '84406B', '85123A'],
        ['21730', '22752', '71053', ..., '84029G', '84406B', '85123A'],
        ['21730', '22752', '71053', ..., '84029G', '84406B', '85123A'],
        ...,
        ['21730', '22752', '71053', ..., '84029G', '84406B', '85123A'],
        ['21730', '22752', '71053', ..., '84029G', '84406B', '85123A'],
        ['21730', '22752', '71053', ..., '84029G', '84406B', '85123A']],

       [['21730', '21730', '21730', ..., '21730', '21730', '21730'],
        ['22752', '22752', '22752', ..., '22752', '22752', '22752'],
        ['71053', '71053', '71053', ..., '71053', '71053', '71053'],
        ...,
        ['84029G', '84029G', '84029G', ..., '84029G', '84029G',
         '84029G'],
        ['84406B', '84406B', '84406B', ..., '84406B', '84406B',
         '84406B'],
        ['85123A', '85123A', '85123A', ..., '85123A', '85123A',
         '85123A']]], dtype='<U6')

**Candedate Generation**

**F_k-1 x F_k-1**  <img src="images/fk-1xfk-1.png"  />

In [231]:
k_itemset =  np.array(L[4])
k_itemset

array([['21730', '22752', '71053', '84029E'],
       ['21730', '22752', '71053', '84029G'],
       ['21730', '22752', '71053', '84406B'],
       ['21730', '22752', '71053', '85123A'],
       ['21730', '22752', '84029E', '84029G'],
       ['21730', '22752', '84029E', '84406B'],
       ['21730', '22752', '84029E', '85123A'],
       ['21730', '22752', '84029G', '84406B'],
       ['21730', '22752', '84029G', '85123A'],
       ['21730', '22752', '84406B', '85123A'],
       ['21730', '71053', '84029E', '84029G'],
       ['21730', '71053', '84029E', '84406B'],
       ['21730', '71053', '84029E', '85123A'],
       ['21730', '71053', '84029G', '84406B'],
       ['21730', '71053', '84029G', '85123A'],
       ['21730', '71053', '84406B', '85123A'],
       ['21730', '84029E', '84029G', '84406B'],
       ['21730', '84029E', '84029G', '85123A'],
       ['21730', '84029E', '84406B', '85123A'],
       ['21730', '84029G', '84406B', '85123A'],
       ['22745', '22748', '22749', '84879'],
       ['22752'

In [232]:
k_2 =  k_itemset[:,:4]
k_2

array([['21730', '22752', '71053', '84029E'],
       ['21730', '22752', '71053', '84029G'],
       ['21730', '22752', '71053', '84406B'],
       ['21730', '22752', '71053', '85123A'],
       ['21730', '22752', '84029E', '84029G'],
       ['21730', '22752', '84029E', '84406B'],
       ['21730', '22752', '84029E', '85123A'],
       ['21730', '22752', '84029G', '84406B'],
       ['21730', '22752', '84029G', '85123A'],
       ['21730', '22752', '84406B', '85123A'],
       ['21730', '71053', '84029E', '84029G'],
       ['21730', '71053', '84029E', '84406B'],
       ['21730', '71053', '84029E', '85123A'],
       ['21730', '71053', '84029G', '84406B'],
       ['21730', '71053', '84029G', '85123A'],
       ['21730', '71053', '84406B', '85123A'],
       ['21730', '84029E', '84029G', '84406B'],
       ['21730', '84029E', '84029G', '85123A'],
       ['21730', '84029E', '84406B', '85123A'],
       ['21730', '84029G', '84406B', '85123A'],
       ['22745', '22748', '22749', '84879'],
       ['22752'

In [233]:
unq,_ = np.unique(k_2, axis=0, return_inverse=True)
cnt = np.bincount(_)
unq = unq.view(k_2.dtype).reshape(-1, k_2.shape[1])

In [234]:
unq =  unq.astype('object')
unq

array([['21730', '22752', '71053', '84029E'],
       ['21730', '22752', '71053', '84029G'],
       ['21730', '22752', '71053', '84406B'],
       ['21730', '22752', '71053', '85123A'],
       ['21730', '22752', '84029E', '84029G'],
       ['21730', '22752', '84029E', '84406B'],
       ['21730', '22752', '84029E', '85123A'],
       ['21730', '22752', '84029G', '84406B'],
       ['21730', '22752', '84029G', '85123A'],
       ['21730', '22752', '84406B', '85123A'],
       ['21730', '71053', '84029E', '84029G'],
       ['21730', '71053', '84029E', '84406B'],
       ['21730', '71053', '84029E', '85123A'],
       ['21730', '71053', '84029G', '84406B'],
       ['21730', '71053', '84029G', '85123A'],
       ['21730', '71053', '84406B', '85123A'],
       ['21730', '84029E', '84029G', '84406B'],
       ['21730', '84029E', '84029G', '85123A'],
       ['21730', '84029E', '84406B', '85123A'],
       ['21730', '84029G', '84406B', '85123A'],
       ['22745', '22748', '22749', '84879'],
       ['22752'

In [235]:
cnt

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [236]:
uniqe_item =  np.column_stack((unq, cnt))
uniqe_item

array([['21730', '22752', '71053', '84029E', 1],
       ['21730', '22752', '71053', '84029G', 1],
       ['21730', '22752', '71053', '84406B', 1],
       ['21730', '22752', '71053', '85123A', 1],
       ['21730', '22752', '84029E', '84029G', 1],
       ['21730', '22752', '84029E', '84406B', 1],
       ['21730', '22752', '84029E', '85123A', 1],
       ['21730', '22752', '84029G', '84406B', 1],
       ['21730', '22752', '84029G', '85123A', 1],
       ['21730', '22752', '84406B', '85123A', 1],
       ['21730', '71053', '84029E', '84029G', 1],
       ['21730', '71053', '84029E', '84406B', 1],
       ['21730', '71053', '84029E', '85123A', 1],
       ['21730', '71053', '84029G', '84406B', 1],
       ['21730', '71053', '84029G', '85123A', 1],
       ['21730', '71053', '84406B', '85123A', 1],
       ['21730', '84029E', '84029G', '84406B', 1],
       ['21730', '84029E', '84029G', '85123A', 1],
       ['21730', '84029E', '84406B', '85123A', 1],
       ['21730', '84029G', '84406B', '85123A', 1],


In [237]:
for i, j in combinations(unq[:6:], 2):
    print(i)

['21730' '22752' '71053' '84029E']
['21730' '22752' '71053' '84029E']
['21730' '22752' '71053' '84029E']
['21730' '22752' '71053' '84029E']
['21730' '22752' '71053' '84029E']
['21730' '22752' '71053' '84029G']
['21730' '22752' '71053' '84029G']
['21730' '22752' '71053' '84029G']
['21730' '22752' '71053' '84029G']
['21730' '22752' '71053' '84406B']
['21730' '22752' '71053' '84406B']
['21730' '22752' '71053' '84406B']
['21730' '22752' '71053' '85123A']
['21730' '22752' '71053' '85123A']
['21730' '22752' '84029E' '84029G']


In [238]:
unq[:6:][1]

array(['21730', '22752', '71053', '84029G'], dtype=object)

In [239]:
apriori_gen(unq)

TypeError: apriori_gen() missing 1 required positional argument: 'order'

In [ ]:
unq[10]

In [ ]:
unq[np.lexsort(unq[::3])]

In [ ]:
a = [ '84029E', '71053']
a.sort(key=lambda x : x.split())
a